## MC
------------

In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

> next part is an example

In [4]:
# functions from the pandas library to record historical index prices from the Stooq platform; the start 
df = wb.DataReader('^SPX', data_source='stooq', start='2010-1-1', end='2021-3-11')

In [5]:
# calculation of logarithmic rates of return, average rate of return and standard deviation and drift
logarithmic_returns = np.log(1 + df['Close'].pct_change())
mu_r = logarithmic_returns.mean()
std_r = logarithmic_returns.std()
drift = mu_r - 0.5*(std_r**2)
wyniki = []

In [ ]:
# function that generates 20,000 rates of return consecutive trading days according to Monte Carlo simulation
for i in range(20000):
    d_returns = np.exp(drift + std_r * norm.ppf(np.random.rand(252)))
    list_prices = [df['Close'][0]]
    # function that calculates the new price after taking into account the generated rate of return from the simulation
    for x in d_returns:
        list_prices.append(list_prices[-1]*x)
    plt.plot(list_prices)
    wyniki.append(list_prices)
    lista=[]
    # A loop that adds the results to the time series to be visualized
    for z in results:
        lista.append(z[-1])

C:\Users\Daniela\AppData\Local\Temp\ipykernel_14472\2799356803.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  list_prices = [df['Close'][0]]


KeyboardInterrupt: 

-----

In [ ]:
"""
Compute the logarithmic returns of Google stock
"""
log_return = np.log(1 + data.pct_change())
#Plot
sns.distplot(log_returns.iloc[1:])
plt.xlabel("Daily Return")
plt.ylabel("Frequency")

logarithmic_returns = np.log(1 + df['Close'].pct_change())
mu_r = logarithmic_returns.mean()
std_r = logarithmic_returns.std()
drift = mu_r - 0.5*(std_r**2)

In [ ]:
#compute drift
u = log_returns.mean()
var = log_returns.var()
drift = u - (0.5*var)

In [ ]:
'''
Compute the Variance and Daily Returns
In this step we have to generate random variables for every day 
forecasted and for every simulation trial we will run

31 dias->Janeiro(valor de previsão final) 
'''
stdev = log_returns.std()
days = 31
trials = 10000
Z = norm.ppf(np.random.rand(days, trials)) #days, trials
daily_returns = np.exp(drift.values + stdev.values * Z)

In [ ]:
''' 
Calculating the stock price for every trial
'''
price_paths = np.zeros_like(daily_returns)
price_paths[0] = data.iloc[-1]
for t in range(1, days):
    price_paths[t] = price_paths[t-1]*daily_returns[t]

''' 
With the price_paths matrix, now you can calculate the probability of profitability, 
or the expected annualized returm, for example. 
'''

In [ ]:
#fazer func
def MC(df, predict_date:31, n_diff_predicts):
    #P1 -> Create simulations to explore a variety of portfolio allocations
    #get returns of stocks
    #fazer log
    log_returns = np.log(1 + df['Close'].pct_change())
    #obter mean and var
    u = log_returns.mean()
    var = log_returns.var()
    drift = u - (0.5*var)

    #deviation between the two, try to understand

    #P2 -> identify the one that optimizes your objective (e.g., maximize return, minimize risk, or optimize the Sharpe ratio)
    


    ...

------------
Automating the Monte Carlo Simulations and Showing CAPM Metrics
------------

----------
References

> https://medium.com/analytics-vidhya/monte-carlo-simulations-for-predicting-stock-prices-python-a64f53585662
> https://journals.ue.wroc.pl/fins/article/download/138/7/458